In [1]:
library(MAST)
library(Seurat)

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tap

ERROR: Error in library(Seurat): there is no package called ‘Seurat’


In [2]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /n/app/openblas/0.2.19/lib/libopenblas_core2p-r0.2.19.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] MAST_1.12.0                 SingleCellExperiment_1.8.0 
 [3] SummarizedExperiment_1.16.1 DelayedArray_0.12.3        
 [5] BiocParallel_1.20.1         matrixStats_0.56.0         
 [7] Biobase_2.46.0              GenomicRanges_1.38.0       
 [9] GenomeInfoDb_1.22.1         IRanges_2.

### Load data

In [ ]:
# data_path = '/n/groups/price/martin/tms_gene_data'
# tissue_data_folder = paste0(data_path, '/adata_by_tissue')
# output_folder = paste0(data_path, '/DGE_result/DGE_bulk')

In [6]:
# data_name = 'facs'
# tissue = 'Aorta'
# celltype = 'aortic endothelial cell'

In [1]:
# Load parameters 
ROOT_FOLDER="/n/groups/price/martin/tms_gene_data"
output_folder=paste0(ROOT_FOLDER, '/toy')
data_name = 'facs'
str_n_gene = 'F'
analyte = 'Aorta.aortic endothelial cell'
keep_sex = 'M' 
flag_3_18 = 'T'
tissue = unlist(strsplit(analyte, "[.]"))[1]
celltype = unlist(strsplit(analyte, "[.]"))[2]
celltype = gsub('_', ' ', celltype)

print(paste0('output_folder: ', output_folder))
print(paste0('data_name: ', data_name))
print(paste0('str_n_gene: ', str_n_gene))
print(paste0('analyte: ', analyte))
print(paste0('keep_sex: ', keep_sex))
print(paste0('flag_3_18: ', flag_3_18))
print(paste0('tissue: ', tissue))
print(paste0('celltype: ', celltype))

[1] "output_folder: /n/groups/price/martin/tms_gene_data/toy"
[1] "data_name: facs"
[1] "str_n_gene: F"
[1] "analyte: Aorta.aortic endothelial cell"
[1] "keep_sex: M"
[1] "FLAG_3_18: T"
[1] "tissue: Aorta"
[1] "celltype: aortic endothelial cell"


In [2]:
# adata_temp = readRDS(paste0(tissue_data_folder, '/bulk.normalized.Bone.rds'))
adata_temp = readRDS(paste0('/n/groups/price/martin/tms_gene_data/rds_by_tissue.1e4/',
                            data_name, '.normalized.', tissue, '.rds'))

In [7]:
sum((adata_temp$age=='3m')|(adata_temp$age=='18m'))

[1] 682

In [ ]:
(adata_temp$age.'')

In [8]:
print('Before filtering')
print(dim(adata_temp))

if (is.na(celltype)==FALSE){
    print('tc analysis')
    ind_select = (adata_temp$cell_ontology_class==celltype)
    adata_temp = adata_temp[,ind_select]
} else {
    print('tissue analysis')
}

if (is.na(keep_sex)==FALSE){
    if (keep_sex=='M'){
        print('keep only male')
        ind_select = (adata_temp$sex=='male')
        adata_temp = adata_temp[,ind_select]
    }
    if (keep_sex=='F'){
        print('keep only female')
        ind_select = (adata_temp$sex=='female')
        adata_temp = adata_temp[,ind_select]
    } 
} else {
    print('keep both sexes')
}

if (is.na(flag_3_18)==FALSE){
    if (flag_3_18=='T'){
        print('keep only 3m and 18m')
        ind_select = (adata_temp$age=='3m')|(adata_temp$age=='18m')
        adata_temp = adata_temp[,ind_select]
    } else {
        print('no age filtering')
    }
} else {
    print('no age filtering')
}

print('After filtering')
print(dim(adata_temp))

[1] "Before filtering"
[1] 22966   906
[1] "tc analysis"
[1] "keep only male"
[1] "keep only 3m and 18m"
[1] "After filtering"
[1] 22966   152


In [8]:
# Prepare sca object
sca <- SceToSingleCellAssay(adata_temp, class = "SingleCellAssay")
colData(sca)$n_genes = scale(colData(sca)$n_genes) # n_gene (CDR)
sca_filt = sca[rowSums(assay(sca)) != 0, ]

`fData` has no primerid.  I'll make something up.

`cData` has no wellKey.  I'll make something up.

Assuming data assay in position 1, with name X is log-transformed.



In [9]:
# Set flags 
if (length(unique(sca_filt$sex))>1){
    flag_sex=TRUE
} else {
    flag_sex=FALSE
}
print(paste0('flag_sex: ', flag_sex))
if (str_n_gene=='T'){
    flag_n_gene=TRUE
} else {
    flag_n_gene=FALSE
}
print(paste0('flag_n_gene: ', flag_n_gene))

[1] "flag_sex: TRUE"
[1] "flag_n_gene: FALSE"


In [10]:
# DGE testing 
covariate = ''
if (flag_sex==TRUE){
    covariate = paste0(covariate, " + sex")
}
if (flag_n_gene==TRUE){
    covariate = paste0(covariate, " + n_genes")
}

print(paste0('covariate: ', covariate))

zlmCond <- zlm(formula = as.formula(paste0("~age_num", covariate)), sca=sca_filt)

summaryCond <- summary(zlmCond, doLRT="age_num")
if (flag_sex==TRUE){
    summaryCond_sex <- summary(zlmCond, doLRT='sexmale')
}

[1] "covariate:  + sex"



 Completed [--------------------------------------------]   0% with 0 failures

 Completed [--------------------------------------------]   1% with 0 failures

 Completed [>-------------------------------------------]   1% with 0 failures

 Completed [>-------------------------------------------]   2% with 0 failures

 Completed [>-------------------------------------------]   3% with 0 failures

 Completed [=>------------------------------------------]   3% with 0 failures

 Completed [=>------------------------------------------]   4% with 0 failures

 Completed [=>------------------------------------------]   5% with 0 failures

 Completed [=>------------------------------------------]   6% with 0 failures

 Completed [==>-----------------------------------------]   6% with 0 failures

 Completed [==>-----------------------------------------]   7% with 0 failures

 Completed [==>-----------------------------------------]   8% with 0 failures

 Completed [===>-----------------------


 Completed [===============================>------------]  74% with 0 failures

 Completed [================================>-----------]  74% with 0 failures

 Completed [================================>-----------]  75% with 0 failures

 Completed [================================>-----------]  76% with 0 failures

 Completed [=================================>----------]  76% with 0 failures

 Completed [=================================>----------]  77% with 0 failures

 Completed [=================================>----------]  78% with 0 failures

 Completed [==================================>---------]  78% with 0 failures

 Completed [==================================>---------]  79% with 0 failures

 Completed [==================================>---------]  80% with 0 failures

 Completed [==================================>---------]  81% with 0 failures

 Completed [===================================>--------]  81% with 0 failures

 Completed [===========================


 Completed [===============>----------------------------]  35% with 0 failures

 Completed [===============>----------------------------]  36% with 0 failures

 Completed [===============>----------------------------]  37% with 0 failures

 Completed [================>---------------------------]  38% with 0 failures

 Completed [================>---------------------------]  39% with 0 failures

 Completed [================>---------------------------]  40% with 0 failures

 Completed [=================>--------------------------]  40% with 0 failures

 Completed [=================>--------------------------]  41% with 0 failures

 Completed [=================>--------------------------]  42% with 0 failures

 Completed [==================>-------------------------]  42% with 0 failures

 Completed [==================>-------------------------]  43% with 0 failures

 Completed [==================>-------------------------]  44% with 0 failures

 Completed [===================>-------

Calculating log-fold changes

Warning message in melt(lfc):
“The melt generic in data.table has been passed a list and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is deprecated, and this redirection is now deprecated as well. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace like reshape2::melt(lfc). In the next version, this warning will become an error.”
Calculating likelihood ratio tests

Refitting on reduced model...


 Completed [--------------------------------------------]   0% with 0 failures

 Completed [--------------------------------------------]   1% with 0 failures

 Completed [>-------------------------------------------]   1% with 0 failures

 Completed [>-------------------------------------------]   2% with 0 failures

 Completed [>-------------------------------------------]   3% with 0 failures

 Completed [=>------------------------------------------]


 Completed [=============================>--------------]  69% with 0 failures

 Completed [==============================>-------------]  69% with 0 failures

 Completed [==============================>-------------]  70% with 0 failures

 Completed [==============================>-------------]  71% with 0 failures

 Completed [==============================>-------------]  72% with 0 failures

 Completed [===============================>------------]  72% with 0 failures

 Completed [===============================>------------]  73% with 0 failures

 Completed [===============================>------------]  74% with 0 failures

 Completed [================================>-----------]  74% with 0 failures

 Completed [================================>-----------]  75% with 0 failures

 Completed [================================>-----------]  76% with 0 failures

 Completed [=================================>----------]  76% with 0 failures

 Completed [===========================

In [136]:
# Summarize results 
summaryDt <- summaryCond$datatable
dt1 = summaryDt[contrast=="age_num" & component=="H", .(primerid, `Pr(>Chisq)`)]
dt2 = summaryDt[contrast=="age_num" & component=="logFC", .(primerid, coef, z)]
de_res = merge(dt1, dt2, by="primerid")
colnames(de_res) <- c("gene", "age.H_p", "age.logFC", 'age.logFC_z')
de_res$age.H_fdr <- p.adjust(de_res$age.H_p, "fdr")

if (flag_sex==TRUE){
    summaryDt <- summaryCond_sex$datatable
    dt_sex1 = summaryDt[contrast=="sexmale" & component=="H", .(primerid, `Pr(>Chisq)`)]
    dt_sex2 = summaryDt[contrast=="sexmale" & component=="logFC", .(primerid, coef, z)]
    de_res_sex = merge(dt_sex1, dt_sex2, by="primerid")
    colnames(de_res_sex) <- c("gene", "sexmale.H_p", "sexmale.logFC", 'sexmale.logFC_z')
    de_res_sex$sexmale.H_fdr <- p.adjust(de_res_sex$sexmale.H_p, "fdr")
    de_res = merge(de_res, de_res_sex, by="gene")
}

# # Write results
# write.csv(de_res,paste0(output_folder, '/',analyte,'.csv'), row.names=FALSE)

print('Finished')

[1] "Finished"


In [140]:
summaryDt <- summaryCond_sex$datatable
dt2 = summaryDt[contrast=="sexmale" & component=="logFC", .(primerid, coef, z)]
dt2

primerid,coef,z
<chr>,<dbl>,<dbl>
0610005C13Rik,NaN,NaN
0610007C21Rik,0.3098507758,3.50291625
0610007L01Rik,0.0025120503,0.03600530
0610007N19Rik,0.0048906780,0.35801808
0610007P08Rik,-0.0316686890,-0.66722572
0610007P14Rik,0.0031783681,0.08529248
0610007P22Rik,0.0456352731,1.33814888
0610009B14Rik,-0.0003094135,-0.01561167
0610009B22Rik,0.0212677159,0.61057467


In [137]:
de_res

gene,age.H_p,age.logFC,age.logFC_z,age.H_fdr,sexmale.H_p,sexmale.logFC,sexmale.logFC_z,sexmale.H_fdr
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0610005C13Rik,0.3034193121,NaN,NaN,0.446676357,0.271953529,NaN,NaN,0.57869073
0610007C21Rik,0.0709517605,-3.592060e-03,-0.91150837,0.200311870,0.001732853,0.3098507758,3.50291625,0.03877487
0610007L01Rik,0.0002275937,-2.140950e-03,-0.54117096,0.002976304,0.237995626,0.0025120503,0.03600530,0.57867076
0610007N19Rik,0.3361448684,-2.889028e-04,-0.20229701,0.481736733,0.780462155,0.0048906780,0.35801808,0.88851801
0610007P08Rik,0.0800414553,-3.166100e-03,-1.02142773,0.219741575,0.009720820,-0.0316686890,-0.66722572,0.11660029
0610007P14Rik,0.8413683852,2.091590e-04,0.10582543,0.895439533,0.589880730,0.0031783681,0.08529248,0.80499554
0610007P22Rik,0.4416476056,3.709536e-04,0.57068970,0.579188303,0.081840644,0.0456352731,1.33814888,0.37996230
0610009B14Rik,0.1359415019,-1.049336e-04,-0.03242321,0.289356332,0.103302005,-0.0003094135,-0.01561167,0.41947488
0610009B22Rik,0.1349376928,9.938030e-04,0.70986667,0.288208379,0.302429625,0.0212677159,0.61057467,0.60600185


In [131]:
de_res

gene,age.raw_p,age.coef,age.coef_z,age.bh_p
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0610005C13Rik,0.3034193121,NaN,NaN,0.446676357
0610007C21Rik,0.0709517605,-3.592060e-03,-0.91150837,0.200311870
0610007L01Rik,0.0002275937,-2.140950e-03,-0.54117096,0.002976304
0610007N19Rik,0.3361448684,-2.889028e-04,-0.20229701,0.481736733
0610007P08Rik,0.0800414553,-3.166100e-03,-1.02142773,0.219741575
0610007P14Rik,0.8413683852,2.091590e-04,0.10582543,0.895439533
0610007P22Rik,0.4416476056,3.709536e-04,0.57068970,0.579188303
0610009B14Rik,0.1359415019,-1.049336e-04,-0.03242321,0.289356332
0610009B22Rik,0.1349376928,9.938030e-04,0.70986667,0.288208379


In [124]:
dt2

primerid,coef,z
<chr>,<dbl>,<dbl>
0610005C13Rik,NaN,NaN
0610007C21Rik,-3.592060e-03,-0.91150837
0610007L01Rik,-2.140950e-03,-0.54117096
0610007N19Rik,-2.889028e-04,-0.20229701
0610007P08Rik,-3.166100e-03,-1.02142773
0610007P14Rik,2.091590e-04,0.10582543
0610007P22Rik,3.709536e-04,0.57068970
0610009B14Rik,-1.049336e-04,-0.03242321
0610009B22Rik,9.938030e-04,0.70986667


In [121]:
de_res

gene,age.raw_p,age.coef,age.coef_z,age.bh_p,sexmale.raw_p,sexmale.coef,sexmale.coef_z,sexmale.bh_p
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0610005C13Rik,0.3034193121,NaN,NaN,0.446676357,0.271953529,NaN,NaN,0.57869073
0610007C21Rik,0.0709517605,-3.592060e-03,-0.91150837,0.200311870,0.001732853,0.3098507758,3.50291625,0.03877487
0610007L01Rik,0.0002275937,-2.140950e-03,-0.54117096,0.002976304,0.237995626,0.0025120503,0.03600530,0.57867076
0610007N19Rik,0.3361448684,-2.889028e-04,-0.20229701,0.481736733,0.780462155,0.0048906780,0.35801808,0.88851801
0610007P08Rik,0.0800414553,-3.166100e-03,-1.02142773,0.219741575,0.009720820,-0.0316686890,-0.66722572,0.11660029
0610007P14Rik,0.8413683852,2.091590e-04,0.10582543,0.895439533,0.589880730,0.0031783681,0.08529248,0.80499554
0610007P22Rik,0.4416476056,3.709536e-04,0.57068970,0.579188303,0.081840644,0.0456352731,1.33814888,0.37996230
0610009B14Rik,0.1359415019,-1.049336e-04,-0.03242321,0.289356332,0.103302005,-0.0003094135,-0.01561167,0.41947488
0610009B22Rik,0.1349376928,9.938030e-04,0.70986667,0.288208379,0.302429625,0.0212677159,0.61057467,0.60600185


In [114]:
summaryDt <- summaryCond$datatable
temp_dt1 = summaryDt[primerid=='Rpl13a', ]
summaryDt <- summaryCond_sex$datatable
temp_dt1_sex = summaryDt[primerid=='Lars2', ]
# temp_dt2_sex = summaryDt[contrast=="age_num" & component=="logFC",]

In [83]:
res_logFC = logFC(zlmCond)

In [96]:
res_logFC$logFC['Lars2',]

age_num     sexmale 
-0.01634798  0.71159469

In [116]:
v_age = sca_filt$age_num

v_sex = sca_filt$sex
ux1 = 0.054550535 + 0.732833076
vx1 = 1 / (1 + exp( 0.007010614 - 1.648932113))
ux0 = 0.732833076
vx0 = 1 / (1 + exp(-1.648932113))
ux1*vx1 - ux0*vx0

[1] 0.04500572

In [115]:
temp_dt1

primerid,component,contrast,Pr(>Chisq),ci.hi,ci.lo,coef,z
<chr>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Rpl13a,C,age_num,1.439283e-35,0.06235387,0.04674720,0.054550535,13.7014613
Rpl13a,C,(Intercept),NA,0.89471682,0.57094933,0.732833076,8.8725798
Rpl13a,C,sexmale,NA,0.30336215,-0.01490934,0.144226405,1.7763360
Rpl13a,D,age_num,6.721926e-01,0.02639488,-0.04041611,-0.007010614,-0.4113261
Rpl13a,D,(Intercept),NA,2.23634454,1.06151969,1.648932113,5.5018372
Rpl13a,D,sexmale,NA,1.52703452,0.34291273,0.934973628,3.0951455
Rpl13a,H,age_num,2.066210e-34,NA,NA,NA,NA
Rpl13a,S,age_num,NA,NA,NA,NA,9.3975447
Rpl13a,S,(Intercept),NA,NA,NA,NA,10.1642477


In [105]:
temp_dt1_sex

primerid,component,contrast,Pr(>Chisq),ci.hi,ci.lo,coef,z
<chr>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Lars2,C,sexmale,1.206457e-24,0.812341174,0.56460892,0.688475047,10.89390887
Lars2,C,(Intercept),NA,0.847367738,0.58979347,0.718580606,10.93581387
Lars2,C,age_num,NA,-0.012309598,-0.02459328,-0.018451440,-5.88816155
Lars2,D,sexmale,4.168522e-03,1.857253014,0.33121558,1.094234299,2.81075651
Lars2,D,(Intercept),NA,2.819836169,1.36342129,2.091628730,5.62960053
Lars2,D,age_num,NA,0.044328472,-0.04212583,0.001101323,0.04993511
Lars2,H,sexmale,2.580572e-25,NA,NA,NA,NA
Lars2,S,sexmale,NA,NA,NA,NA,9.69066182
Lars2,S,(Intercept),NA,NA,NA,NA,11.71351685


In [46]:
help(crossprod)

In [51]:
v1 = 1:4
v2 = c(1,1,1,1)

In [53]:
crossprod(v1, v2)

10


### Old code

In [ ]:
if (is.na(cell_type)==FALSE){
    ind_select = (adata_temp$cell_ontology_class==cell_type)
    adata_temp = adata_temp[,ind_select]
}

In [ ]:
adata_temp

In [ ]:
sum(adata_temp$cell_ontology_class==cell_type)

In [ ]:
sca <- SceToSingleCellAssay(adata_temp, class = "SingleCellAssay")
# n_gene (CDR)
colData(sca)$n_genes = scale(colData(sca)$n_genes)
sca_filt = sca[rowSums(assay(sca)) != 0, ]

In [ ]:
if (length(unique(sca_filt$sex))){
    flag_sex=TRUE
} else {
    flag_sex=FALSE
}

In [ ]:
covariate = ''
if (flag_sex==TRUE){
    covariate = paste0(covariate, " + sex")
}
if (flag_n_gene==TRUE){
    covariate = paste0(covariate, " + n_genes")
}


In [ ]:
zlmCond <- zlm(formula = as.formula(paste0("~age_num", covariate)), sca=sca_filt)

In [ ]:
if (flag_sex==TRUE){
    summaryCond_sex <- summary(zlmCond, doLRT='sexmale')
} else {
    summaryCond <- summary(zlmCond, doLRT="age_num")
}

In [ ]:
summaryDt <- summaryCond$datatable
dt1 = summaryDt[contrast=="age_num" & component=="H", .(primerid, `Pr(>Chisq)`)]
dt2 = summaryDt[contrast=="age_num" & component=="logFC", .(primerid, coef)]
de_res = merge(dt1, dt2, by="primerid")
colnames(de_res) <- c("gene", "age.raw_p", "age.coef")
de_res$age.bh_p <- p.adjust(de_res$age.raw_p, "fdr")

In [ ]:
if (flag_sex==TRUE){
    summaryDt <- summaryCond_sex$datatable
    dt_sex1 = summaryDt[contrast=="sexmale" & component=="H", .(primerid, `Pr(>Chisq)`)]
    dt_sex2 = summaryDt[contrast=="sexmale" & component=="logFC", .(primerid, coef)]
    de_res_sex = merge(dt_sex1, dt_sex2, by="primerid")
    colnames(de_res_sex) <- c("gene", "sexmale.raw_p", "sexmale.coef")
    de_res_sex$sexmale.bh_p <- p.adjust(de_res_sex$sexmale.raw_p, "fdr")
    de_res = merge(de_res, de_res_sex, by="gene")
}

In [ ]:
output_folder

In [ ]:
tissue='Bone'

In [ ]:
output_folder

In [ ]:
write.csv(de_res,paste0(output_folder, '/',tissue,'.csv'), 
          row.names=FALSE)




In [ ]:
head(de_res)